# Export temperature at atmospheric levels
- This script is used to export temperature at atmospheric levels
- Simulations: GM_SLUCM, GM_CLMU, GM_CLMU_ALB, GM_CLMU_NOAH. 

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from wrf import getvar, interplevel, latlon_coords, vertcross
home_path = "/gws/nopw/j04/duicv/yuansun/"

In [2]:
# --- Configuration ---
model_list = ['wrf', 'wrf-ctsm', 'wrf-ctsm_uniform_urban', 'wrf-ctsm_road_albedo'] # , 'wrf-ctsm_road_albedo_init'
#model_list = ['wrf', 'wrf-ctsm', 'wrf-ctsm_road_albedo']
var_list = ['T', 'pressure', 'z']  # example variables
center_i, center_j = 30, 18        # adjust to your location indices
season_dates = {
    'summer': ['07-16', '07-17', '07-18', '07-19'], #
    'winter': ['01-04', '01-05', '01-06']
}
day_hour, night_hour = 13, 1  # daytime and nighttime hours

# --- Function to get WRF filename based on model, season, date ---
def get_ncfile(model, season, date):
    if season == 'winter':
        if model in ['wrf', 'wrf-ctsm']:
            return f"{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/wrfout_d01_2022-01-04_00:00:00.nc"
        elif model == 'wrf-ctsm_uniform_urban':
            return f"{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/uniform_urban_noheating/wrfout_d01_2021-12-25_00:00:00.nc"
        elif model == 'wrf-ctsm_road_albedo':
            return f"{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo/wrfout_d01_2021-12-25_00:00:00.nc"
        #elif model == 'wrf-ctsm_road_albedo_init':
            #return None # no winter run for this model
    elif season == 'summer':
        if model == 'wrf-ctsm':
            return f"{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/wrfout_d01_2022-07-15_01:00:00.nc"
        elif model == 'wrf-ctsm_road_albedo':
            return f"{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo_hw/wrfout_d01_2022-07-16_01:00:00.nc"
        #elif model == 'wrf-ctsm_road_albedo_init':    
            #return f"{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo_july/wrfout_d01_2022-07-07_00:00:00.nc"
        elif model == 'wrf':
            # custom logic for wrf summer dates
            if date in ['07-16','07-17']:
                return f"{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/wrfout_d01_2022-07-13_01:00:00.nc"
            else:
                return f"{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/wrfout_d01_2022-07-18_01:00:00.nc"
        elif model == 'wrf-ctsm_uniform_urban':
             return f"{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/uniform_urban_noac_restart/wrfout_d01_2022-07-11_00:00:00.nc"
# --- Function to find nearest time index in WRF ---
def find_time_index(nc_time, target_time):
    return int(np.argmin(np.abs(nc_time - np.datetime64(target_time))))

# --- Loop over models, seasons, and dates ---
time_list = []

for model in model_list:
    for season, dates in season_dates.items():
        if model == 'wrf-ctsm_road_albedo_init' and season == 'winter':
            continue
        for date in dates:
            ncfile_path = get_ncfile(model, season, date)
            ncfile = Dataset(ncfile_path)
            times = pd.to_datetime(xr.open_dataset(ncfile_path)['XTIME'].values)

            # define day/night datetime
            day_time = pd.to_datetime(f'2022-{date} {day_hour}:00:00')
            night_time = pd.to_datetime(f'2022-{date} {night_hour}:00:00')

            idx_day = find_time_index(times, day_time)
            idx_night = find_time_index(times, night_time)

            # Extract variables for day and night
            day_vals = [getvar(ncfile, var, timeidx=idx_day)[:, center_j, center_i].values for var in var_list]
            night_vals = [getvar(ncfile, var, timeidx=idx_night)[:, center_j, center_i].values for var in var_list]
            # Build DataFrame
            df_rows = pd.DataFrame([day_vals, night_vals], columns=var_list)
            df_rows['model'] = model
            df_rows['location'] = 'sp'
            df_rows['tag'] = ['13:30', '01:30']
            df_rows['date'] = date
            df_rows['season'] = season
            time_list.append(df_rows)

# --- Combine all rows ---
df_time = pd.concat(time_list, ignore_index=True)

# --- Convert potential temperature to actual temperature ---
df_time['T_actual'] = (df_time['T'] + 300) * (df_time['pressure'] / 1000) ** 0.286 - 273.15

# --- Compute mean vertical profile ---
df_time_mean = df_time.groupby(['model', 'location', 'tag', 'season'])[['T_actual', 'z']].mean().reset_index()

# --- Save for plotting ---
df_time_mean.to_pickle('./data_for_figure/vertical_profile_mean.pkl')

print(df_time_mean.head())

      model location    tag  season  \
0       wrf       sp  01:30  summer   
1       wrf       sp  01:30  winter   
2       wrf       sp  13:30  summer   
3       wrf       sp  13:30  winter   
4  wrf-ctsm       sp  01:30  summer   

                                            T_actual  \
0  [21.26413, 20.93077, 20.872887, 20.796371, 20....   
1  [1.3989054, 1.8916117, 2.1004333, 1.9257101, 1...   
2  [28.79673, 28.162888, 27.450676, 26.569382, 25...   
3  [3.5905254, 3.199941, 2.6629333, 1.9523112, 1....   
4  [20.251488, 20.19294, 20.55365, 20.69291, 20.6...   

                                                   z  
0  [66.194824, 107.69847, 166.20601, 250.33618, 3...  
1  [64.40795, 102.97422, 157.47218, 235.84749, 31...  
2  [66.79568, 109.285484, 169.05344, 254.83347, 3...  
3  [64.58607, 103.40021, 158.03664, 236.47389, 31...  
4  [66.11952, 107.509895, 165.9388, 250.01392, 33...  
